In [5]:
import refinitiv.data as rd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
import pandas as pd
from hmmlearn.hmm import GaussianHMM,GMMHMM, CategoricalHMM
from sklearn.decomposition import PCA
import plotly.graph_objects as go
from plotly.graph_objs.scatter.marker import Line
from plotly.subplots import make_subplots
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from sklearn_extra.cluster import KMedoids
from scipy.stats import wasserstein_distance
from sklearn.mixture import GaussianMixture
import pickle
import warnings
import math

warnings.filterwarnings('ignore')

In [6]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x107c1ebe0 {name='workspace'}>

## Data Ingestion and Engineering

In [7]:
class DataEngineering:
    
    def __init__(self, prices, split_date):
        self.prices = prices
        self.split_date = split_date
        self.instrument = prices.columns.name
    
    def prepare_data(self):
        prices_change = self.calculate_change_of_ma(self.prices)
        data = self.prepare_data_for_model_input(prices_change)
        split_index = self.get_split_index(prices_change, self.split_date)
        
        return prices_change.set_index('Date'), data, split_index
    
    def calculate_change_of_ma(self, prices):

        prices_ma = prices.rolling(5).mean()
        
        prices_change = prices_ma.pct_change()
        prices_change[f'{self.instrument}_close'] = prices[self.instrument]
        prices_change.dropna(inplace=True)
        return prices_change

    def prepare_data_for_model_input(self, prices_change):
        data_dict = {}
        for column in prices_change.columns:
            if column != f'{self.instrument}_close':
                data_dict[column] = np.array([[q] for q in prices_change[column].values])
        data = np.column_stack(data_dict.values())
        return data
    
    def split_for_train_test(self, data, split_index):

        rets_train = data[:split_index]
        rets_test = data[split_index:]
        
        return rets_train, rets_test
    
    def get_split_index(self, prices_change, split_date):
        prices_change.reset_index(inplace = True)
        split_index = prices_change.loc[prices_change['Date'] > split_date].iloc[0].name
        
        return split_index

## Modeling and Evaluation

In [10]:
class RegimeDetection:
    
    def get_regimes_hmm(self, input_data):
        hmm_model = GaussianHMM(
        n_components=2,  n_iter=10000, covariance_type="full", random_state = 1000)
        return hmm_model.fit(input_data)
    
    def get_regimes_clustering(self, model):
        if model == 'AgglomerativeClustering':
            clustering = AgglomerativeClustering(n_clusters = 2, linkage = 'complete',  affinity = 'manhattan')
        elif model == 'kmeans':
            clustering = KMeans(n_clusters=2)
        
        return clustering
    
    def get_regimes_gmm(self, input_data):
        gmm = GaussianMixture(n_components=2, covariance_type = 'full', max_iter = 100000, n_init =30,
                    init_params = 'random_from_data').fit(input_data)
        return gmm
            
    # def initialise_object(self, obj, params):
    #     for k, v in params.items():
    #         setattr(obj, k, v)

        return obj

In [9]:
def plot_hidden_states(hidden_states, df, n_components):

    colors = ['blue', 'green', 'yellow', 'black', 'grey']
    for i in range(n_components):
        mask = hidden_states == i
        print('Number of observations for State ', i,":", len(df.index[mask]))
        
        fig = go.Figure()
        fig.add_trace(go.Line(x=df.index, y=df[f"{prices_change.columns.name}_close"],
                    name = f'Price {prices_change.columns.name}',
                    line_color = 'red'))
        
        fig.add_trace(go.Scatter(x=df.index[mask], y=df[f"{prices_change.columns.name}_close"][mask],
                    mode='markers',
                    name='Hidden State ' + str(i)))
        
        fig.update_traces(marker=dict(size=4,color=colors[i]),
                  selector=dict(mode='markers'))
        
        fig.update_layout(height=400, width=900, legend=dict(
            yanchor="top", y=0.99,
            xanchor="left",x=0.01), 
            margin=dict(l=20, r=20, t=20, b=20))
        
        fig.show()

In [2]:
def plot_hidden_states_cf(hidden_states, df, n_components):

    colors = ['blue', 'green', 'yellow', 'black', 'grey']
    for i in range(n_components):
        mask = hidden_states == i
        print('Number of observations for State ', i,":", len(df.index[mask]))
        
        fig = go.Figure()
        fig.add_trace(go.Line(x=df.index, y=df[f"close"],
                    name = f'Price',
                    line_color = 'red'))
        
        fig.add_trace(go.Scatter(x=df.index[mask], y=df[f"close"][mask],
                    mode='markers',
                    name='Hidden State ' + str(i)))
        
        fig.update_traces(marker=dict(size=4,color=colors[i]),
                  selector=dict(mode='markers'))
        
        fig.update_layout(height=400, width=900, legend=dict(
            yanchor="top", y=0.99,
            xanchor="right",x=1), 
            margin=dict(l=20, r=20, t=20, b=20))
        
        fig.show()

In [ ]:
def feed_forward_training(model, prices, split_index, train_chunk_size):
    
    models = {'hmm': regime_detection.get_regimes_hmm, 
              'gmm': regime_detection.get_regimes_gmm}
    
    init_train_data = prices[:split_index]
    test_data = prices[split_index:]
    
    states_pred = []
    rd_model = models[model](init_train_data)
    print('hmm score initial training', rd_model.score(init_train_data))
    
    for i in range(math.ceil(len(test_data))):
        split_index += 1
        preds = rd_model.predict(prices[:split_index]).tolist()
        states_pred.append(preds[-1])
        
        if i % train_chunk_size == 0:
            rd_model = models[model](prices[:split_index])
            print(i, 'hmm retrain', rd_model.score(prices[:split_index]))
            
    filename = 'finalized_model_.sav'
    pickle.dump(rd_model, open(filename, 'wb'))
    
    return states_pred

## Strategy Implementation

In [ ]:
class StrategyImplementation:
    

    def run(self):
        activate = False
        first_day = self.trading_data.index[0]
        for index, row in self.trading_data.iterrows():
            self.hmms.append(row['State'])
            is_state_confirmed = self.is_state_confirmed(self.hmms, self.state_confirm_window)
            self.add_trading_day_info(self.trading_activity, index, row)
            if (row['Signal'] != 1 and activate == False) or index == first_day:
                self.remain_idle(self.trading_activity, index, row)
            else:
                activate = True
            if activate:
                if self.is_buy_signal(row, is_state_confirmed):
                    self.buy(self.trading_activity, index, row)
                elif self.is_sell_signal(row, is_state_confirmed):
                    self.sell(self.trading_activity, index, row)
                    activate = False
                elif self.trading_activity['Stock Balance'][-1]==1:
                    self.hold(self.trading_activity, index, row)
                else:
                    self.remain_idle(self.trading_activity, index, row)
        return pd.DataFrame(self.trading_activity).set_index('Date')

                
    def remain_idle(self, trading_activity, index, row):
        trading_activity['Position'].append('Idle')
        trading_activity['Stock Balance'].append(0)
        trading_activity['Stock Value'].append(0)
        if len(trading_activity['Stock Balance']) == 1:
            trading_activity['Cash Balance'].append(0)
            trading_activity['Portfolio Valuation'].append(0)
            trading_activity['P&L'].append(0)  
        else:
            trading_activity['Cash Balance'].append(trading_activity['Cash Balance'][-1])
            trading_activity['Portfolio Valuation'].append(trading_activity['Stock Value'][-1]+trading_activity['Cash Balance'][-1])
            trading_activity['P&L'].append(trading_activity['P&L'][-1])
            
    def buy(self, trading_activity, index, row):
        trading_activity['Position'].append('Buy')
        trading_activity['Stock Balance'].append(trading_activity['Stock Balance'][-1] + 1)
        trading_activity['Cash Balance'].append(trading_activity['Cash Balance'][-1]-(trading_activity['Current Price'][-1]))
        trading_activity['Stock Value'].append(trading_activity['Stock Balance'][-1] * trading_activity['Current Price'][-1])
        trading_activity['Portfolio Valuation'].append(trading_activity['Stock Value'][-1]+trading_activity['Cash Balance'][-1])
        trading_activity['P&L'].append(trading_activity['P&L'][-1])  
            
    def sell(self, trading_activity, index, row):
        trading_activity['Position'].append('Sell')
        trading_activity['Cash Balance'].append(trading_activity['Cash Balance'][-1] + (trading_activity['Current Price'][-1] * trading_activity['Stock Balance'][-1]))
        trading_activity['Stock Balance'].append(0)
        trading_activity['Stock Value'].append(0)
        trading_activity['Portfolio Valuation'].append(trading_activity['Stock Value'][-1]+trading_activity['Cash Balance'][-1])
        trading_activity['P&L'].append(trading_activity['Cash Balance'][-1]) 
        activate = False

    def is_state_confirmed(self, states, state_confirm_window):
        is_state_confirmed = True
        if state_confirm_window !=0:
            states = states[-state_confirm_window:]
            is_state_confirmed = states.count(states[0]) == len(states)
        return is_state_confirmed
    
    def hold(self, trading_activity, index, row):
        trading_activity['Position'].append('Hold')
        trading_activity['Stock Balance'].append(trading_activity['Stock Balance'][-1])
        trading_activity['Cash Balance'].append(trading_activity['Cash Balance'][-1])
        trading_activity['Stock Value'].append(trading_activity['Stock Balance'][-1] * trading_activity['Current Price'][-1])
        trading_activity['Portfolio Valuation'].append(trading_activity['Stock Value'][-1]+trading_activity['Cash Balance'][-1])
        trading_activity['P&L'].append(trading_activity['P&L'][-1])            


        

In [16]:
class MovingaAveragesStrategy(StrategyImplementation):
    def __init__(self, prices,  fast_ma, slow_ma, hmm_constraint = True, state_confirm_window=2):
        self.prices = prices
        self.fast_ma = fast_ma
        self.slow_ma = slow_ma
        self.hmm_constraint = hmm_constraint
        self.state_confirm_window = state_confirm_window
        self.instrument = prices.columns.name
        self.hmms = []
        self.hmm_states = self.prices[['Date', 'State']]
        self.trading_data = self.get_trading_data()
        self.up_state = pd.DataFrame(self.trading_data)['State'].mode()[0]        
        self.trading_activity = {'Date': [],'Current Price': [],f'{self.instrument}_fast_ma':[], f'{self.instrument}_slow_ma':[], 
                                 'Crossover':[], 'Signal':[], 'State':[],
                                 'Position': [], 'Stock Balance': [], 'Cash Balance': [],  
                                 'Stock Value': [],'Portfolio Valuation': [], 'P&L': []}
    
    def get_trading_data(self):
        
        prices_fast_ma = self.calculate_ma(self.fast_ma, 'fast_ma')
        prices_slow_ma = self.calculate_ma(self.slow_ma, 'slow_ma')
        trading_data = self.get_trading_signals(prices_fast_ma, prices_slow_ma)
        trading_data.columns.name = self.instrument
        
        return trading_data
    
    def calculate_ma(self, ma, ma_type):
        prices_ma = pd.DataFrame(self.prices['Date'])
        prices_ma[f'{self.instrument}_{ma_type}'] = self.prices[self.instrument].rolling(ma).mean()
        prices_ma.dropna(inplace = True)
    
        return pd.DataFrame(prices_ma)
    
    def get_trading_signals(self, prices_fast_ma, prices_slow_ma):
        trading_data = self.prices.merge(
                    (prices_fast_ma.merge(prices_slow_ma, on = 'Date')), on = 'Date')
        trading_data.insert(loc = len(trading_data.columns), column = 'Crossover', 
                            value = np.where(trading_data[f'{self.instrument}_fast_ma'] > trading_data[f'{self.instrument}_slow_ma'],1,0))
        trading_data['Signal'] = trading_data['Crossover'].diff()
        
        return trading_data.set_index('Date')
    
    def is_sell_signal(self, row, is_state_confirmed):
        sell_signal = False
        if self.trading_activity['Stock Balance'][-1] != 0:
            sell_signal = row['Signal'] == -1
            if self.hmm_constraint:
                sell_signal = sell_signal or (row['State'] != self.up_state and is_state_confirmed == True)
            
        return sell_signal
    
    def is_buy_signal(self, row, is_state_confirmed):
        buy_signal = False
        if self.trading_activity['Stock Balance'][-1] == 0:
            buy_signal =  row['Signal'] == 1
            if self.hmm_constraint:
                buy_signal = (buy_signal or row['ESc1_slow_ma'] < row['ESc1_fast_ma']) and row['State'] == self.up_state and is_state_confirmed == True
        return buy_signal
    
    def add_trading_day_info(self, trading_activity, index, row):
        trading_activity['Date'].append(index)
        trading_activity[f'{self.instrument}_fast_ma'].append(row[f'{self.instrument}_fast_ma'])
        trading_activity[f'{self.instrument}_slow_ma'].append(row[f'{self.instrument}_slow_ma'])
        trading_activity['Crossover'].append(row['Crossover'])
        trading_activity['Signal'].append(row['Signal'])
        trading_activity['State'].append(row['State'])
        trading_activity['Current Price'].append(row[self.instrument])

In [12]:
class HMMStrategy(StrategyImplementation):
    def __init__(self, trading_data, state_confirm_window=2):
        self.prices = prices
        self.state_confirm_window = state_confirm_window
        self.instrument = prices.columns.name
        self.trading_data = self.get_trading_data()
        self.up_state = pd.DataFrame(self.trading_data)['State'].mode()[0]
        self.hmms = []
        self.trading_activity = {'Date': [],'Current Price': [], 'Signal':[], 'State':[],
                                 'Position': [], 'Stock Balance': [], 'Cash Balance': [],  
                                 'Stock Value': [],'Portfolio Valuation': [], 'P&L': []}

    def get_trading_data(self):
        trading_data = self.prices.copy()
        trading_data.insert(loc = len(trading_data.columns), column = 'Signal', 
                            value = np.where(trading_data['State'] == trading_data['State'].mode()[0],1,0))
        
        return trading_data.set_index('Date')

    
    def is_sell_signal(self, row, is_state_confirmed):
        sell_signal = False
        if self.trading_activity['Stock Balance'][-1] != 0:
            sell_signal = row['State'] != self.up_state and is_state_confirmed == True
        return sell_signal
    
    def is_buy_signal(self, row, is_state_confirmed):
        buy_signal = False
        if self.trading_activity['Stock Balance'][-1] == 0:
            buy_signal = row['State'] == self.up_state and is_state_confirmed == True
        return buy_signal

    
    def add_trading_day_info(self, trading_activity, index, row):
        trading_activity['Date'].append(index)
        trading_activity['Signal'].append(row['Signal'])
        trading_activity['State'].append(row['State'])
        trading_activity['Current Price'].append(row[self.instrument])

In [ ]:
class StrategyOutcomePlot:
    
    def __init__(self, strategy_outcome):
        self.strategy_outcome = strategy_outcome
        self.secondary_axis = {'Current Price': False, 'P&L': False, 'Portfolio Valuation': True}
        
    def plot(self, outcome, signal_dot):
        
        self.add_signal_dots(strategy_outcome, signal_dot)
        
        fig = make_subplots(specs=[[{"secondary_y": True}]])
        
        fig.add_trace(go.Line(x=strategy_outcome.index, y=strategy_outcome["Current Price"], name = 'Price SPY',
                    line_color = 'blue'))
        fig.add_trace(go.Line(x= strategy_outcome.index, y=strategy_outcome[outcome], name = outcome,
                    line_color = 'green'),  secondary_y=False)
        
        fig.add_trace(go.Scatter(x= strategy_outcome.index, y=strategy_outcome["Buy Signal"],name='Buy Signal',
                    mode='markers'), secondary_y = self.secondary_axis[signal_dot])
        fig.add_trace(go.Scatter(x= strategy_outcome.index, y=strategy_outcome["Sell Signal"], name='Sell Signal',
                    mode='markers', line_color = 'red'), secondary_y = self.secondary_axis[signal_dot])
        
        fig.update_layout(height=400, width=900, legend=dict(
            yanchor="top", y=0.99,
            xanchor="left",x=0.01), 
            margin=dict(l=20, r=20, t=20, b=20))

        fig.show()
    
    def add_signal_dots(self, strategy_outcome, signal_dot):
        strategy_outcome['Buy Signal'] = np.where(strategy_outcome['Position'] =='Buy',  strategy_outcome[signal_dot],np.nan)
        strategy_outcome['Sell Signal'] = np.where(strategy_outcome['Position'] =='Sell',  strategy_outcome[signal_dot],np.nan)
